In [1]:
import pyspark
import pyspark.sql.functions as func
import pyspark.sql.types as types
import pyspark
spark = pyspark.sql.SparkSession.builder \
    .appName("houseStatsDf") \
    .getOrCreate()
sc = spark.sparkContext

In [42]:
tmp = spark.read.format("csv") \
         .option("header", "true") \
         .option("mode", "DROPMALFORMED") \
         .load('data/Bike-Sharing-Dataset/hour.csv')

In [43]:
tmp

DataFrame[instant: string, dteday: string, season: string, yr: string, mnth: string, hr: string, holiday: string, weekday: string, workingday: string, weathersit: string, temp: string, atemp: string, hum: string, windspeed: string, casual: string, registered: string, cnt: string]

In [44]:
tmp.take(5)

[Row(instant='1', dteday='2011-01-01', season='1', yr='0', mnth='1', hr='0', holiday='0', weekday='6', workingday='0', weathersit='1', temp='0.24', atemp='0.2879', hum='0.81', windspeed='0', casual='3', registered='13', cnt='16'),
 Row(instant='2', dteday='2011-01-01', season='1', yr='0', mnth='1', hr='1', holiday='0', weekday='6', workingday='0', weathersit='1', temp='0.22', atemp='0.2727', hum='0.8', windspeed='0', casual='8', registered='32', cnt='40'),
 Row(instant='3', dteday='2011-01-01', season='1', yr='0', mnth='1', hr='2', holiday='0', weekday='6', workingday='0', weathersit='1', temp='0.22', atemp='0.2727', hum='0.8', windspeed='0', casual='5', registered='27', cnt='32'),
 Row(instant='4', dteday='2011-01-01', season='1', yr='0', mnth='1', hr='3', holiday='0', weekday='6', workingday='0', weathersit='1', temp='0.24', atemp='0.2879', hum='0.75', windspeed='0', casual='3', registered='10', cnt='13'),
 Row(instant='5', dteday='2011-01-01', season='1', yr='0', mnth='1', hr='4', h

In [45]:
type(tmp)

pyspark.sql.dataframe.DataFrame

In [46]:
tmp.printSchema()
tmp.show()

root
 |-- instant: string (nullable = true)
 |-- dteday: string (nullable = true)
 |-- season: string (nullable = true)
 |-- yr: string (nullable = true)
 |-- mnth: string (nullable = true)
 |-- hr: string (nullable = true)
 |-- holiday: string (nullable = true)
 |-- weekday: string (nullable = true)
 |-- workingday: string (nullable = true)
 |-- weathersit: string (nullable = true)
 |-- temp: string (nullable = true)
 |-- atemp: string (nullable = true)
 |-- hum: string (nullable = true)
 |-- windspeed: string (nullable = true)
 |-- casual: string (nullable = true)
 |-- registered: string (nullable = true)
 |-- cnt: string (nullable = true)

+-------+----------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
|instant|    dteday|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|
+-------+----------+------+---+----+---+-------+-------+----------+----------+----+------+---

In [48]:
tmp_GB = tmp.groupBy('dteday','season') \
    .agg(func.avg('temp').alias('avgTemp'),
         func.avg('atemp').alias('avgAtemp'),
         func.avg('hum').alias('avgHum'),
         func.avg('windspeed').alias('avgWindspeed')) \
    .orderBy('dteday','season')
tmp_GB.show()

+----------+------+-------------------+-------------------+-------------------+-------------------+
|    dteday|season|            avgTemp|           avgAtemp|             avgHum|       avgWindspeed|
+----------+------+-------------------+-------------------+-------------------+-------------------+
|2011-01-01|     1|0.34416666666666673|0.36362500000000003| 0.8058333333333335| 0.1604458333333333|
|2011-01-02|     1|0.36347826086956525|0.35373913043478267| 0.6960869565217392|0.24853913043478262|
|2011-01-03|     1|0.19636363636363632|0.18940454545454546|0.43727272727272726| 0.2483090909090908|
|2011-01-04|     1|0.19999999999999998|0.21212173913043478| 0.5904347826086958|  0.160295652173913|
|2011-01-05|     1|0.22695652173913042| 0.2292695652173913| 0.4369565217391305|             0.1869|
|2011-01-06|     1|0.20434782608695654|0.23320869565217398| 0.5182608695652174|0.08956521739130435|
|2011-01-07|     1|0.19652173913043483| 0.2088391304347826| 0.4986956521739131|0.16872608695652172|


In [49]:
tmp_GB.createTempView('stat')

In [58]:
spark.sql("""
SELECT * FROM stat
WHERE season <> 1
""").show()

+----------+------+-------------------+-------------------+-------------------+-------------------+
|    dteday|season|            avgTemp|           avgAtemp|             avgHum|       avgWindspeed|
+----------+------+-------------------+-------------------+-------------------+-------------------+
|2011-03-21|     2| 0.4304347826086957|0.40973478260869556| 0.7373913043478261|0.28878260869565225|
|2011-03-22|     2|0.44166666666666665|0.44064166666666665| 0.6245833333333333|0.22575000000000003|
|2011-03-23|     2| 0.3469565217391305| 0.3379391304347827| 0.8395652173913042|0.23426086956521738|
|2011-03-24|     2| 0.2849999999999999| 0.2708333333333333| 0.8058333333333328|0.24378749999999996|
|2011-03-25|     2| 0.2641666666666666|0.25631249999999994| 0.4950000000000001|0.23072499999999999|
|2011-03-26|     2|0.26583333333333337|0.25757083333333336| 0.3941666666666667|0.20957083333333335|
|2011-03-27|     2| 0.2530434782608695| 0.2503391304347826|0.49391304347826087|             0.1843|


In [59]:
spark.catalog.listTables()

[Table(name='stat', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]